# Build Machine Learning Model
## Objective
### The project aim is to determine the culverts that need maintenance(due to change in precipitation) based on the culvert rating and this is achieved by building a machine learning model to predict culvert rating 

### Import required packages

In [1]:
# for exploring loaded datasets
import pandas as pd 
import numpy as np

# import scikitlearn machine learning model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier

# imports of tensorflow model
from tensorflow.keras import models, datasets, layers, optimizers

# import train_test_split function from scikitlearn for dataset splitting
from sklearn.model_selection import train_test_split

# import functions from scikitlearn for model evaluation
from sklearn.metrics import mean_squared_error,  accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix
from imblearn.metrics import specificity_score, geometric_mean_score

# for performing a grid search over specified parameter values for a given estimator
from sklearn.model_selection import GridSearchCV   

#For ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# from google.colab import files
# uploaded = files.upload()

### Loading Required datasets

In [3]:
# load the datasets one by one
# CDOT_Culvert_data
CDOT_Culvert_data = pd.read_excel("CDOT_Culvert data.xlsx")
CDOT_Culvert_data.head()

,BRKEY,FEATINT,FHWA_REGN,DISTRICT,COUNTY,FACILITY,LOCATION,CUSTODIAN,OWNER,BB_PCT,...,structtype,asphaltthi,INSPDATE,SCOURCRIT,DKRATING,SUPRATING,SUBRATING,CHANRATING,CULVRATING,MERRA_ID
0,050B331430BR,UNNAMED DRAINAGE,8,24,101,US50 ML,4.3 MI W OF JCT SH 209,1,1,-1,...,CBC,72,2019-05-03,5,N,N,N,6,7,141818
1,040A196350BL,UNNAMED DRAINAGE,8,36,49,US40 ML,5.1 MI W OF HOT SULFUR SP,1,1,-1,...,CMP,48,2020-07-07,8,N,N,N,7,7,144119
2,085G000060BL,UNNAMED DRAINAGE,8,41,123,US 85 BUSS RT,0.4 MI S OF JCT US 34 ML,1,1,-1,...,CBC,60,2017-12-08,8,N,N,N,7,6,144697
3,159A032090BR,SANGRE DE CRISTO DITCH#3,8,57,23,SH159 ML,1.57 MI S OF JCT US160,1,1,0,...,CMP,36,2021-12-16,8,N,N,N,4,6,141240
4,007D068280BR,STANLEY DITCH,8,15,14,SH 7 ML,0.1 MI W OF JCT I-25,1,1,-1,...,CBC,120,2018-01-02,8,N,N,N,7,6,144121


In [4]:
# from google.colab import files
# uploaded = files.upload()

In [5]:
# MerraClimate_Dat_Subset
MerraClimate_Dat_Subset = pd.read_excel("MerraClimate_Dat_Subset.xlsx")


# select  2020 data
MerraClimate_Dat_Subset_2020 = MerraClimate_Dat_Subset.loc[MerraClimate_Dat_Subset['YEAR'] == 2020]

MerraClimate_Dat_Subset_2020

,MERRA_ID,ELEVATION,YEAR,MONTH,TEMP_AVG,PRECIPITATION,REL_HUM_AVG_AVG,WIND_VELOCITY_AVG,PERCENT_SUNSHINE_AVG
153,140659,1561,2020,3,6.4,29.767,55,2,49.6
154,140659,1561,2020,2,1.2,9.042,52,3,63.4
164,140659,1561,2020,4,11.1,5.846,40,3,71.3
169,140659,1561,2020,1,-0.6,17.144,62,2,58.1
172,140659,1561,2020,9,19.4,9.951,30,2,88.3
...,...,...,...,...,...,...,...,...,...
18715,145277,1143,2020,8,26.1,7.583,42,3,73.7
18716,145277,1143,2020,7,26.0,42.244,51,3,64.3
18717,145277,1143,2020,12,0.6,22.438,54,3,54.9
18718,145277,1143,2020,11,6.2,2.535,46,3,73.8


### Datasets Merging

In [6]:
#  merge MerraClimate_Dat_Subset_2020 to CDOT_Culvert_data

# Let's merge MerraClimate_Dat_Subset_2020 to CDOT_Culvert_data on MERRA_ID
merged_data = pd.merge(MerraClimate_Dat_Subset_2020, CDOT_Culvert_data, on='MERRA_ID')

merged_data.head()

,MERRA_ID,ELEVATION,YEAR,MONTH,TEMP_AVG,PRECIPITATION,REL_HUM_AVG_AVG,WIND_VELOCITY_AVG,PERCENT_SUNSHINE_AVG,BRKEY,...,ADTYEAR,structtype,asphaltthi,INSPDATE,SCOURCRIT,DKRATING,SUPRATING,SUBRATING,CHANRATING,CULVRATING
0,140659,1561,2020,3,6.4,29.767,55,2,49.6,160A026270BR,...,2007,CMP,48,2021-03-23,5,N,N,N,6,6
1,140659,1561,2020,3,6.4,29.767,55,2,49.6,160A011010BR,...,2007,CMP,48,2021-03-18,5,N,N,N,5,7
2,140659,1561,2020,3,6.4,29.767,55,2,49.6,160A023140BR,...,2007,CMP,120,2021-03-23,4,N,N,N,5,6
3,140659,1561,2020,3,6.4,29.767,55,2,49.6,160A025310BR,...,2007,CMP,30,2021-03-23,8,N,N,N,6,7
4,140659,1561,2020,3,6.4,29.767,55,2,49.6,160A030110BR,...,2007,CMP,36,2021-03-23,8,N,N,N,7,7


In [7]:
#get info of our dataframe
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71484 entries, 0 to 71483
Data columns (total 53 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MERRA_ID              71484 non-null  int64         
 1   ELEVATION             71484 non-null  int64         
 2   YEAR                  71484 non-null  int64         
 3   MONTH                 71484 non-null  int64         
 4   TEMP_AVG              71484 non-null  float64       
 5   PRECIPITATION         71484 non-null  float64       
 6   REL_HUM_AVG_AVG       71484 non-null  int64         
 7   WIND_VELOCITY_AVG     71484 non-null  int64         
 8   PERCENT_SUNSHINE_AVG  71484 non-null  float64       
 9   BRKEY                 71484 non-null  object        
 10  FEATINT               71484 non-null  object        
 11  FHWA_REGN             71484 non-null  int64         
 12  DISTRICT              71484 non-null  int64         
 13  COUNTY          

In [8]:
#get summary statistics for data
merged_data.describe()

,MERRA_ID,ELEVATION,YEAR,MONTH,TEMP_AVG,PRECIPITATION,REL_HUM_AVG_AVG,WIND_VELOCITY_AVG,PERCENT_SUNSHINE_AVG,FHWA_REGN,...,ON_UNDER,KIND_HWY,LEVL_SRVC,KMPOST,ADTTOTAL,ADTYEAR,asphaltthi,SCOURCRIT,CHANRATING,CULVRATING
count,71484.000000,71484.000000,71484.0,71484.000000,71484.000000,71484.000000,71484.000000,71484.000000,71484.000000,71484.0,...,71484.0,71484.000000,71484.000000,71484.000000,71484.000000,71484.000000,71484.000000,71484.000000,71484.000000,71484.000000
mean,142909.659057,2165.253651,2020.0,6.500000,8.443446,43.381607,48.076129,2.304194,64.898478,8.0,...,1.0,2.317442,1.045828,112.202101,8902.899110,1995.301662,77.010240,7.257680,6.333893,6.136814
std,1264.129262,705.730448,0.0,3.452077,10.088833,31.987561,12.387738,0.946865,8.641216,0.0,...,0.0,0.739173,0.518017,112.227745,16085.608141,162.018380,69.126552,1.716508,1.801607,1.235942
min,140659.000000,1003.000000,2020.0,1.000000,-13.400000,1.918000,22.000000,0.000000,43.500000,8.0,...,1.0,0.000000,0.000000,0.000000,-1.000000,-4.000000,-1.000000,0.000000,0.000000,0.000000
25%,141817.000000,1590.000000,2020.0,3.750000,0.000000,20.938000,39.000000,2.000000,59.100000,8.0,...,1.0,2.000000,1.000000,25.769999,1100.000000,2007.000000,36.000000,8.000000,6.000000,6.000000
50%,142969.000000,1974.000000,2020.0,6.500000,8.000000,34.808000,46.000000,2.000000,64.100000,8.0,...,1.0,2.000000,1.000000,71.010000,3100.000000,2008.000000,60.000000,8.000000,7.000000,6.000000
75%,144116.000000,2629.000000,2020.0,9.250000,16.800000,54.882000,55.000000,3.000000,70.900000,8.0,...,1.0,3.000000,1.000000,164.210000,9000.000000,2010.000000,96.000000,8.000000,7.000000,7.000000
max,145277.000000,3906.000000,2020.0,12.000000,28.100000,280.061000,91.000000,5.000000,88.500000,8.0,...,1.0,4.000000,8.000000,496.579989,193000.000000,2020.000000,960.000000,9.000000,8.000000,8.000000


In [9]:
#Check the dataset dimensions
print('Shape of dataset:  ' , merged_data.shape)

Shape of dataset:   (71484, 53)


### Data Preparation

#### To make a machine learning model to predict culvert rating, we first need to prepare the data by selecting the relevant features, handling missing values and encoding categorical variables.

In [10]:
#Select relevant features

features = ['YEARBUILT', 'SERVTYPON', 'SERVTYPUND', 'MAINSPANS', 'APPSPANS', 'MAXSPAN', 'LENGTH','MATERIALMA', 'DESIGNMAIN', 'ADTTOTAL', 'ADTYEAR', 'structtype', 'asphaltthi', 'ELEVATION', 'TEMP_AVG', 'PRECIPITATION', 'REL_HUM_AVG_AVG', 'WIND_VELOCITY_AVG', 'PERCENT_SUNSHINE_AVG', 'BB_PCT', 'SCOURCRIT', 'CHANRATING', 'CULVRATING']

In [11]:
selected_df = merged_data[features]

In [12]:
selected_df

,YEARBUILT,SERVTYPON,SERVTYPUND,MAINSPANS,APPSPANS,MAXSPAN,LENGTH,MATERIALMA,DESIGNMAIN,ADTTOTAL,...,ELEVATION,TEMP_AVG,PRECIPITATION,REL_HUM_AVG_AVG,WIND_VELOCITY_AVG,PERCENT_SUNSHINE_AVG,BB_PCT,SCOURCRIT,CHANRATING,CULVRATING
0,1960,1,5,1,0,7.001,7.001,3,19,5000,...,1561,6.4,29.767,55,2,49.6,0,5,6,6
1,2012,1,5,1,0,5.801,5.801,3,19,2200,...,1561,6.4,29.767,55,2,49.6,0,5,5,7
2,1970,1,5,1,0,12.999,12.999,3,19,5000,...,1561,6.4,29.767,55,2,49.6,0,4,5,6
3,1960,1,5,1,0,3.999,3.999,3,19,5000,...,1561,6.4,29.767,55,2,49.6,0,8,6,7
4,1970,1,5,1,0,7.001,7.001,3,19,6900,...,1561,6.4,29.767,55,2,49.6,0,8,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71479,1968,1,5,1,0,7.999,7.999,1,19,6800,...,1143,9.0,13.877,48,3,61.2,0,8,7,6
71480,1968,1,0,1,0,11.499,11.499,1,19,6900,...,1143,9.0,13.877,48,3,61.2,0,8,0,6
71481,1968,1,5,1,0,5.000,5.000,1,19,6900,...,1143,9.0,13.877,48,3,61.2,0,8,7,7
71482,2000,1,5,3,0,4.501,15.499,3,19,690,...,1143,9.0,13.877,48,3,61.2,0,8,7,6


In [13]:
#Check for null

selected_df.isnull().sum()

YEARBUILT               0
SERVTYPON               0
SERVTYPUND              0
MAINSPANS               0
APPSPANS                0
MAXSPAN                 0
LENGTH                  0
MATERIALMA              0
DESIGNMAIN              0
ADTTOTAL                0
ADTYEAR                 0
structtype              0
asphaltthi              0
ELEVATION               0
TEMP_AVG                0
PRECIPITATION           0
REL_HUM_AVG_AVG         0
WIND_VELOCITY_AVG       0
PERCENT_SUNSHINE_AVG    0
BB_PCT                  0
SCOURCRIT               0
CHANRATING              0
CULVRATING              0
dtype: int64

In [14]:
# # Check for duplicates

selected_df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
71479    False
71480    False
71481    False
71482    False
71483    False
Length: 71484, dtype: bool

### Create dummy variables for catagorical columns

In [15]:
# Find unique values in a column
unique_values = selected_df['structtype'].unique()

print(unique_values)

['CMP' 'CBC' 'RCPC' 'TBC' 'CPP' 'SAC' 'TTS' 'RAC' 'PCBC' 'CAC' 'CS' 'CI'
 'AAC' 'CRF' 'TSLAB' 'TS' ' ' 'SSM' 'CDTPG' 'TTM']


In [16]:
#Create dummy variables

df_with_dummy_variables = pd.get_dummies(selected_df, columns=['structtype'], prefix=['structtype'], prefix_sep='_')

In [17]:
df_with_dummy_variables.columns

Index(['YEARBUILT', 'SERVTYPON', 'SERVTYPUND', 'MAINSPANS', 'APPSPANS',
       'MAXSPAN', 'LENGTH', 'MATERIALMA', 'DESIGNMAIN', 'ADTTOTAL', 'ADTYEAR',
       'asphaltthi', 'ELEVATION', 'TEMP_AVG', 'PRECIPITATION',
       'REL_HUM_AVG_AVG', 'WIND_VELOCITY_AVG', 'PERCENT_SUNSHINE_AVG',
       'BB_PCT', 'SCOURCRIT', 'CHANRATING', 'CULVRATING', 'structtype_ ',
       'structtype_AAC', 'structtype_CAC', 'structtype_CBC',
       'structtype_CDTPG', 'structtype_CI', 'structtype_CMP', 'structtype_CPP',
       'structtype_CRF', 'structtype_CS', 'structtype_PCBC', 'structtype_RAC',
       'structtype_RCPC', 'structtype_SAC', 'structtype_SSM', 'structtype_TBC',
       'structtype_TS', 'structtype_TSLAB', 'structtype_TTM',
       'structtype_TTS'],
      dtype='object')

In [18]:
#Display df_with_dummies

df_with_dummy_variables.head()

,YEARBUILT,SERVTYPON,SERVTYPUND,MAINSPANS,APPSPANS,MAXSPAN,LENGTH,MATERIALMA,DESIGNMAIN,ADTTOTAL,...,structtype_PCBC,structtype_RAC,structtype_RCPC,structtype_SAC,structtype_SSM,structtype_TBC,structtype_TS,structtype_TSLAB,structtype_TTM,structtype_TTS
0,1960,1,5,1,0,7.001,7.001,3,19,5000,...,0,0,0,0,0,0,0,0,0,0
1,2012,1,5,1,0,5.801,5.801,3,19,2200,...,0,0,0,0,0,0,0,0,0,0
2,1970,1,5,1,0,12.999,12.999,3,19,5000,...,0,0,0,0,0,0,0,0,0,0
3,1960,1,5,1,0,3.999,3.999,3,19,5000,...,0,0,0,0,0,0,0,0,0,0
4,1970,1,5,1,0,7.001,7.001,3,19,6900,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Drop two columns

columns_to_drop = ['structtype_RAC']

In [20]:
#Create new dataframe

df = df_with_dummy_variables.drop(columns_to_drop, axis=1)

In [21]:
df.columns

Index(['YEARBUILT', 'SERVTYPON', 'SERVTYPUND', 'MAINSPANS', 'APPSPANS',
       'MAXSPAN', 'LENGTH', 'MATERIALMA', 'DESIGNMAIN', 'ADTTOTAL', 'ADTYEAR',
       'asphaltthi', 'ELEVATION', 'TEMP_AVG', 'PRECIPITATION',
       'REL_HUM_AVG_AVG', 'WIND_VELOCITY_AVG', 'PERCENT_SUNSHINE_AVG',
       'BB_PCT', 'SCOURCRIT', 'CHANRATING', 'CULVRATING', 'structtype_ ',
       'structtype_AAC', 'structtype_CAC', 'structtype_CBC',
       'structtype_CDTPG', 'structtype_CI', 'structtype_CMP', 'structtype_CPP',
       'structtype_CRF', 'structtype_CS', 'structtype_PCBC', 'structtype_RCPC',
       'structtype_SAC', 'structtype_SSM', 'structtype_TBC', 'structtype_TS',
       'structtype_TSLAB', 'structtype_TTM', 'structtype_TTS'],
      dtype='object')

In [22]:
df.head()

,YEARBUILT,SERVTYPON,SERVTYPUND,MAINSPANS,APPSPANS,MAXSPAN,LENGTH,MATERIALMA,DESIGNMAIN,ADTTOTAL,...,structtype_CS,structtype_PCBC,structtype_RCPC,structtype_SAC,structtype_SSM,structtype_TBC,structtype_TS,structtype_TSLAB,structtype_TTM,structtype_TTS
0,1960,1,5,1,0,7.001,7.001,3,19,5000,...,0,0,0,0,0,0,0,0,0,0
1,2012,1,5,1,0,5.801,5.801,3,19,2200,...,0,0,0,0,0,0,0,0,0,0
2,1970,1,5,1,0,12.999,12.999,3,19,5000,...,0,0,0,0,0,0,0,0,0,0
3,1960,1,5,1,0,3.999,3.999,3,19,5000,...,0,0,0,0,0,0,0,0,0,0
4,1970,1,5,1,0,7.001,7.001,3,19,6900,...,0,0,0,0,0,0,0,0,0,0


### Data Partitioning

In [23]:
# Split data into features and target

X = df.drop('CULVRATING', axis=1)
y = df['CULVRATING']

In [24]:
# Split data into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
X

,YEARBUILT,SERVTYPON,SERVTYPUND,MAINSPANS,APPSPANS,MAXSPAN,LENGTH,MATERIALMA,DESIGNMAIN,ADTTOTAL,...,structtype_CS,structtype_PCBC,structtype_RCPC,structtype_SAC,structtype_SSM,structtype_TBC,structtype_TS,structtype_TSLAB,structtype_TTM,structtype_TTS
0,1960,1,5,1,0,7.001,7.001,3,19,5000,...,0,0,0,0,0,0,0,0,0,0
1,2012,1,5,1,0,5.801,5.801,3,19,2200,...,0,0,0,0,0,0,0,0,0,0
2,1970,1,5,1,0,12.999,12.999,3,19,5000,...,0,0,0,0,0,0,0,0,0,0
3,1960,1,5,1,0,3.999,3.999,3,19,5000,...,0,0,0,0,0,0,0,0,0,0
4,1970,1,5,1,0,7.001,7.001,3,19,6900,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71479,1968,1,5,1,0,7.999,7.999,1,19,6800,...,0,0,0,0,0,0,0,0,0,0
71480,1968,1,0,1,0,11.499,11.499,1,19,6900,...,0,0,0,0,0,0,0,0,0,0
71481,1968,1,5,1,0,5.000,5.000,1,19,6900,...,0,0,0,0,0,0,0,0,0,0
71482,2000,1,5,3,0,4.501,15.499,3,19,690,...,0,0,0,0,0,0,0,0,0,0


In [26]:
y

0        6
1        7
2        6
3        7
4        7
        ..
71479    6
71480    6
71481    7
71482    6
71483    6
Name: CULVRATING, Length: 71484, dtype: int64

### Scaling

In [27]:
#Import the scaling library

from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object

scaler = StandardScaler()

In [28]:
# Fit the scaler to the training data and transform it

X_train_scaled = scaler.fit_transform(X_train)

In [29]:
# Apply the same scaler transformation to the test data

X_test_scaled = scaler.transform(X_test)

### Model Building

##### 1. Random Forest Classifier

In [30]:
# Create a Random Forest classifier object
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

In [31]:
# Train the Random Forest classifier
rf_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test_scaled)

In [32]:
# Evaluate the performance of the classifier

print("Confusion Matrix:\n\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Calculate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Calculate precision, recall, and f1 score
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

# Calculate specificity

cm = confusion_matrix(y_test, y_pred)
tn = cm[0, 0]
fp = cm[0, 1]
specificity = tn / (tn + fp)
print("Specificity:", specificity)

# Calculate negative predictive value

print("Negative Predictive Value:", (confusion_matrix(y_test, y_pred)[0, 0] / (confusion_matrix(y_test, y_pred)[0, 0] + confusion_matrix(y_test, y_pred)[1, 0])))

# Calculate Matthew correlation coefficient
print("Matthew Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))

# Calculate geometric mean
print("Geometric Mean:", geometric_mean_score(y_test, y_pred, average='weighted'))

Confusion Matrix:

 [[ 266    0    0    0    0    0    0    0]
 [   0   47    0    0    0    0    0    0]
 [   0    0  559    0    1    6    1    4]
 [   0    0    1  490    0    4    9    0]
 [   0    0    6    0 2471   73   19    0]
 [   0    0    3    0   45 8099  187    0]
 [   0    0    0    1    3  173 7968   26]
 [   0    0    1    0    0    2   42  939]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       266
           2       1.00      1.00      1.00        47
           3       0.98      0.98      0.98       571
           4       1.00      0.97      0.98       504
           5       0.98      0.96      0.97      2569
           6       0.97      0.97      0.97      8334
           7       0.97      0.98      0.97      8171
           8       0.97      0.95      0.96       984

    accuracy                           0.97     21446
   macro avg       0.98      0.98      0.98     21446
weighted avg  

##### 2. CART

In [33]:
# Create a CART object
cart = DecisionTreeClassifier(random_state=42)

# Train the CART
cart.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = cart.predict(X_test)

In [34]:
# Evaluate the performance of the classifier

print("Confusion Matrix:\n\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Calculate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Calculate precision, recall, and f1 score
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

# Calculate specificity

cm = confusion_matrix(y_test, y_pred)
tn = cm[0, 0]
fp = cm[0, 1]
specificity = tn / (tn + fp)
print("Specificity:", specificity)

# Calculate negative predictive value

print("Negative Predictive Value:", (confusion_matrix(y_test, y_pred)[0, 0] / (confusion_matrix(y_test, y_pred)[0, 0] + confusion_matrix(y_test, y_pred)[1, 0])))

# Calculate Matthew correlation coefficient
print("Matthew Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))

# Calculate geometric mean
print("Geometric Mean:", geometric_mean_score(y_test, y_pred, average='weighted'))

Confusion Matrix:

 [[ 266    0    0    0    0    0    0    0]
 [   0   47    0    0    0    0    0    0]
 [   0    0  570    0    1    0    0    0]
 [   0    0    0  503    0    0    1    0]
 [   0    0    7    0 2499   52   11    0]
 [   0    0    0    2   69 8157  106    0]
 [   0    0    0    2    5  134 8007   23]
 [   0    0    0    0    0    0   30  954]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       266
           2       1.00      1.00      1.00        47
           3       0.99      1.00      0.99       571
           4       0.99      1.00      1.00       504
           5       0.97      0.97      0.97      2569
           6       0.98      0.98      0.98      8334
           7       0.98      0.98      0.98      8171
           8       0.98      0.97      0.97       984

    accuracy                           0.98     21446
   macro avg       0.99      0.99      0.99     21446
weighted avg  

##### 3. Naive BAyes Classifier

In [35]:
# Create a NB object
nb = GaussianNB()

#Train the model
nb.fit(X_train, y_train)

#Making predictions
y_pred = nb.predict(X_test)

In [36]:
# Evaluate the performance of the classifier

print("Confusion Matrix:\n\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Calculate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Calculate precision, recall, and f1 score
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

# Calculate specificity

cm = confusion_matrix(y_test, y_pred)
tn = cm[0, 0]
fp = cm[0, 1]
specificity = tn / (tn + fp)
print("Specificity:", specificity)

# Calculate negative predictive value

print("Negative Predictive Value:", (confusion_matrix(y_test, y_pred)[0, 0] / (confusion_matrix(y_test, y_pred)[0, 0] + confusion_matrix(y_test, y_pred)[1, 0])))

# Calculate Matthew correlation coefficient
print("Matthew Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))

# Calculate geometric mean
print("Geometric Mean:", geometric_mean_score(y_test, y_pred, average='weighted'))

Confusion Matrix:

 [[ 256    3    0    0    0    0    4    3]
 [   0   33    0    0   14    0    0    0]
 [   3  125    0    0  430    8    0    5]
 [   6   72    5    3  396    7    5   10]
 [   2  250    2   15 2165   54   60   21]
 [  37  457    2    6 6916  377  380  159]
 [  63  427    3   15 6313  629  561  160]
 [  18   20    1    2  503  277   75   88]]

Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.96      0.79       266
           2       0.02      0.70      0.05        47
           3       0.00      0.00      0.00       571
           4       0.07      0.01      0.01       504
           5       0.13      0.84      0.22      2569
           6       0.28      0.05      0.08      8334
           7       0.52      0.07      0.12      8171
           8       0.20      0.09      0.12       984

    accuracy                           0.16     21446
   macro avg       0.24      0.34      0.17     21446
weighted avg  

##### 4. KNNClassifier

In [37]:
# Create a KNN object
knn = KNeighborsClassifier(n_neighbors=5)

# Train the KNN classifier
knn.fit(X_train_scaled, y_train)

# Make predictions on the testing set
y_pred = knn.predict(X_test_scaled)

In [38]:
# Evaluate the performance of the classifier

print("Confusion Matrix:\n\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Calculate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Calculate precision, recall, and f1 score
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

# Calculate specificity

cm = confusion_matrix(y_test, y_pred)
tn = cm[0, 0]
fp = cm[0, 1]
specificity = tn / (tn + fp)
print("Specificity:", specificity)

# Calculate negative predictive value

print("Negative Predictive Value:", (confusion_matrix(y_test, y_pred)[0, 0] / (confusion_matrix(y_test, y_pred)[0, 0] + confusion_matrix(y_test, y_pred)[1, 0])))

# Calculate Matthew correlation coefficient
print("Matthew Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))

# Calculate geometric mean
print("Geometric Mean:", geometric_mean_score(y_test, y_pred, average='weighted'))

Confusion Matrix:

 [[ 266    0    0    0    0    0    0    0]
 [   0   19    3    1   12    5    7    0]
 [   0    1  211   11   95  178   74    1]
 [   0    2   36  118   97  175   75    1]
 [   0    3  100   37  934 1015  476    4]
 [   0    3  107   48  730 5293 2123   30]
 [   0    3   67   31  484 2749 4745   92]
 [   0    0    4    8   49  204  364  355]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       266
           2       0.61      0.40      0.49        47
           3       0.40      0.37      0.38       571
           4       0.46      0.23      0.31       504
           5       0.39      0.36      0.38      2569
           6       0.55      0.64      0.59      8334
           7       0.60      0.58      0.59      8171
           8       0.73      0.36      0.48       984

    accuracy                           0.56     21446
   macro avg       0.59      0.49      0.53     21446
weighted avg  

##### 5. Neural Netwroks MLPClassifier

In [39]:
#Creating the MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(16,), activation='relu', solver='adam', random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

In [40]:
# Evaluate the performance of the classifier

print("Confusion Matrix:\n\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Calculate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Calculate precision, recall, and f1 score
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

# Calculate specificity

cm = confusion_matrix(y_test, y_pred)
tn = cm[0, 0]
fp = cm[0, 1]
specificity = tn / (tn + fp)
print("Specificity:", specificity)

# Calculate negative predictive value

print("Negative Predictive Value:", (confusion_matrix(y_test, y_pred)[0, 0] / (confusion_matrix(y_test, y_pred)[0, 0] + confusion_matrix(y_test, y_pred)[1, 0])))

# Calculate Matthew correlation coefficient
print("Matthew Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))

# Calculate geometric mean
print("Geometric Mean:", geometric_mean_score(y_test, y_pred, average='weighted'))

Confusion Matrix:

 [[ 266    0    0    0    0    0    0    0]
 [   0   10    4    0    6   18    9    0]
 [   0    2   73    0   40  353  103    0]
 [   0    0   27   10   61  335   71    0]
 [   0    1   34    0  292 1752  487    3]
 [   0    0   12    0  211 5943 2133   35]
 [   0    3    6    0   84 3678 4247  153]
 [   0    0    1    0    0  202  428  353]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       266
           2       0.62      0.21      0.32        47
           3       0.46      0.13      0.20       571
           4       1.00      0.02      0.04       504
           5       0.42      0.11      0.18      2569
           6       0.48      0.71      0.58      8334
           7       0.57      0.52      0.54      8171
           8       0.65      0.36      0.46       984

    accuracy                           0.52     21446
   macro avg       0.65      0.38      0.41     21446
weighted avg  

##### 6. Latent Dirichlet Allocation

In [41]:
# Create a LDA object
lda = LinearDiscriminantAnalysis()

# Train the LDA model
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

# Train the random forest classifier on the LDA-transformed data
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train_lda, y_train)

# Make predictions on the testing set
y_pred = rfc.predict(X_test_lda)

In [42]:
# Evaluate the performance of the classifier

print("Confusion Matrix:\n\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Calculate accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Calculate precision, recall, and f1 score
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))

# Calculate specificity

cm = confusion_matrix(y_test, y_pred)
tn = cm[0, 0]
fp = cm[0, 1]
specificity = tn / (tn + fp)
print("Specificity:", specificity)

# Calculate negative predictive value

print("Negative Predictive Value:", (confusion_matrix(y_test, y_pred)[0, 0] / (confusion_matrix(y_test, y_pred)[0, 0] + confusion_matrix(y_test, y_pred)[1, 0])))

# Calculate Matthew correlation coefficient
print("Matthew Correlation Coefficient:", matthews_corrcoef(y_test, y_pred))

# Calculate geometric mean
print("Geometric Mean:", geometric_mean_score(y_test, y_pred, average='weighted'))

Confusion Matrix:

 [[ 266    0    0    0    0    0    0    0]
 [   0   26    0    2    2   12    5    0]
 [   0    0  263    6   48  172   80    2]
 [   0    1    6  205   39  167   86    0]
 [   0    0   35    9 1120  928  472    5]
 [   0    0   30    9  272 6118 1883   22]
 [   0    0   20   17  166 2082 5794   92]
 [   0    0    2    3    8  144  280  547]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       266
           2       0.96      0.55      0.70        47
           3       0.74      0.46      0.57       571
           4       0.82      0.41      0.54       504
           5       0.68      0.44      0.53      2569
           6       0.64      0.73      0.68      8334
           7       0.67      0.71      0.69      8171
           8       0.82      0.56      0.66       984

    accuracy                           0.67     21446
   macro avg       0.79      0.61      0.67     21446
weighted avg  